# Stitch
---

#### Overview
Interactive stitching of one or several sections.

In [1]:
# indirectly enable autocomplete
%config Completer.use_jedi = False

# autoreload modules
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
import requests

from tqdm.notebook import tqdm
import renderapi

from scripted_render_pipeline import basic_auth
from interactive_render import plotting

#### `render-ws` environment variables

In [13]:
# create an authorized session
auth = basic_auth.load_auth()
sesh = requests.Session()
sesh.auth = auth

project = "20240222_RL019_hoechst_exocrine"
# render-ws environment variables
params_render = {
    "host": "http://localhost",
    "port": 8081,
    "client_scripts": "/home/catmaid/render/render-ws-java-client/src/main/scripts",
    "client_script": "/home/catmaid/render/render-ws-java-client/src/main/scripts/run_ws_client.sh",
    "owner": "skaracoban",
    "project": project,
    "session": sesh
}

params_align = {
    "host": "http://localhost",
    "port": 8081,
    "client_scripts": "/home/catmaid/render/render-ws-java-client/src/main/scripts",
    "client_script": "/home/catmaid/render/render-ws-java-client/src/main/scripts/run_ws_client.sh",
    "owner": "skaracoban",
    "project": project,
    "memGB": '2G', # Allocate enough memory
    "session": sesh
}

# set project directory
dir_project = pathlib.Path("/long_term_storage/skaracoban/data/test_data/20240222_RL019_hoechst_exocrine")

# set max_workers (for multithreading)
max_workers = 20

## Stitching (part I)
---
Get tile pairs. Get point matches.

### Get tile pairs

Use render-ws `tilePairClient` to get the set of tile pairs.

In [4]:
# choose stack from which to get tile pairs
stack = "EM_himag"
z_values = [int(z) for z in renderapi.stack.get_z_values_for_stack(
    stack,
    **params_render
)]

# Get tile pairs from the rough aligned stack
tilepairs = renderapi.client.tilePairClient(
    stack=stack,
    minz=min(z_values),
    maxz=max(z_values),
    zNeighborDistance=0,  # half-height of search cylinder
    excludeSameLayerNeighbors=False,
    subprocess_mode="check_output",  # suppresses output
    **params_render
)["neighborPairs"]

# Reformat for PointMatchClient
tile_pairs_reformat = [[tp["p"]["id"], tp["q"]["id"]] for tp in tilepairs]

# Get relative positions for first tile in every tilepair
relativePositions = [tp['p']['relativePosition'] for tp in tilepairs]

# Show tile pairs
out = f"Number of tile pairs... {len(tile_pairs_reformat)}"
print(out, "\n" + "-"*len(out))
tile_pairs_reformat[:5]

Number of tile pairs... 264 
---------------------------


[['aaa_aaa_EM_himag-S001-00000x00011', 'aax_aax_EM_himag-S001-00000x00010'],
 ['aaa_aaa_EM_himag-S001-00000x00011', 'aab_aab_EM_himag-S001-00001x00011'],
 ['aab_aab_EM_himag-S001-00001x00011', 'aaw_aaw_EM_himag-S001-00001x00010'],
 ['aab_aab_EM_himag-S001-00001x00011', 'aac_aac_EM_himag-S001-00002x00011'],
 ['aac_aac_EM_himag-S001-00002x00011', 'aav_aav_EM_himag-S001-00002x00010']]

### Get point matches

Use `render-ws` `PointMatchClient` script to find matching features between the neighboring megafields.

In [5]:
from renderapi.client import (
    SiftPointMatchOptions,
    MatchDerivationParameters,
    FeatureExtractionParameters
)

In [6]:
# Name for pointmatch collection
match_collection = f"{params_render['project']}_{stack}_stitch_matches"
match_collection

'20240222_RL019_hoechst_exocrine_EM_himag_stitch_matches'

In [7]:
# renderapi.pointmatch.delete_collection(match_collection,
#                                        **params_render)

#### Set SIFT + RANSAC parameters

In [8]:
# `RANSAC` parameters
params_RANSAC = MatchDerivationParameters(
    matchIterations=None,
    matchMaxEpsilon=25,        # maximal alignment error
    matchMaxNumInliers=None,
    matchMaxTrust=None,
    matchMinInlierRatio=0.05,  # minimal inlier ratio
    matchMinNumInliers=7,      # minimal number of inliers
    matchModelType='AFFINE',   # expected transformation
    matchRod=0.92              # closest/next closest ratio
)

# `SIFT` parameters
params_SIFT = FeatureExtractionParameters(
    SIFTfdSize=8,              # feature descriptor size
    SIFTmaxScale=0.20,         # (width/height *) maximum image size
    SIFTminScale=0.05,         # (width/height *) minimum image size
    SIFTsteps=5                # steps per scale octave
)

# Combined `SIFT` & `RANSAC` parameters
params_SIFT = SiftPointMatchOptions(
    fillWithNoise=True,
    **{**params_RANSAC.__dict__,
       **params_SIFT.__dict__}
)

# Extra parameters
params_SIFT.numberOfThreads = 1  # multithreading
params_SIFT.clipWidth = 640      # N pixels included in rendered clips of LEFT/RIGHT oriented montage tiles
params_SIFT.clipHeight = 640     # N pixels included in rendered clips of TOP/BOTTOM oriented montage tiles
params_SIFT.__dict__

{'SIFTfdSize': 8,
 'SIFTmaxScale': 0.2,
 'SIFTminScale': 0.05,
 'SIFTsteps': 5,
 'matchIterations': None,
 'matchMaxEpsilon': 25,
 'matchMaxNumInliers': None,
 'matchMaxTrust': None,
 'matchMinInlierRatio': 0.05,
 'matchMinNumInliers': 7,
 'matchModelType': 'AFFINE',
 'matchRod': 0.92,
 'renderScale': None,
 'fillWithNoise': True,
 'numberOfThreads': 1,
 'clipWidth': 640,
 'clipHeight': 640}

#### Execute SIFT + RANSAC on `N` cores
Runs `N` parallel threads with one thread per tile pair

In [9]:
import concurrent.futures

In [11]:
futures = set()
all_sections = {}
executor = concurrent.futures.ThreadPoolExecutor(
max_workers = max_workers
)

try:
    for tile_pair, pos in tqdm(zip(tile_pairs_reformat, relativePositions),
                               desc="submitting tile pairs",
                               total=len(tile_pairs_reformat),
                               unit="tilepairs",
                               smoothing=0.3):
        params_SIFT.firstCanvasPosition = pos
        future = executor.submit(
            renderapi.client.pointMatchClient,
                stack=stack,
                collection=match_collection,
                tile_pairs=[tile_pair],
                sift_options=params_SIFT,
                excludeAllTransforms=True,
                subprocess_mode='check_output',  # suppresses output
                **params_align)
        futures.add(future)

    for future in tqdm(
        concurrent.futures.as_completed(futures),
        desc="extracting point matches",
        total=len(futures),
        unit="tilepairs",
        smoothing=min(100 / len(futures), 0.3),
    ):
        futures.remove(future)
        
finally:
    for future in futures:
        future.cancel()
    executor.shutdown()

submitting tile pairs:   0%|          | 0/264 [00:00<?, ?tilepairs/s]

extracting point matches:   0%|          | 0/264 [00:00<?, ?tilepairs/s]

### Inspect

In [10]:
plotting.plot_stack_with_stitching_matches(
    stack,
    match_collection,
    params_render,
    width=256
)

grid shape: (10, 10)


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

## Stitching (part II)
---

### Create alignment files

In [11]:
from pathlib import Path
import os
import subprocess
import json
from pprint import pprint

In [14]:
z_values = renderapi.stack.get_z_values_for_stack(stack,
                                                  **params_render)
# Load align.json template
template_stitch_json = Path('../templates/montage.json')
with template_stitch_json.open('r') as json_data:
    params_stitch = json.load(json_data)

# Edit BigFeta solver schema
params_stitch['first_section'] = min(z_values)
params_stitch['last_section'] = max(z_values)
params_stitch['transformation'] = 'TranslationModel'

# Edit input stack data
params_stitch['input_stack']['host'] = params_render['host']
params_stitch['input_stack']['owner'] = params_render['owner']
params_stitch['input_stack']['project'] = params_render['project']
params_stitch['input_stack']['name'] = stack

# Edit point match stack data
params_stitch['pointmatch']['host'] = params_render['host']
params_stitch['pointmatch']['owner'] = params_render['owner']
params_stitch['pointmatch']['name'] = match_collection

# Edit output stack data
params_stitch['output_stack']['host'] = params_render['host']
params_stitch['output_stack']['owner'] = params_render['owner']
params_stitch['output_stack']['project'] = params_render['project']
params_stitch['output_stack']['name'] = f"{stack}_stitched"

# Edit regularization parameters
params_stitch['regularization']['default_lambda'] = 0.005      # default: 0.005
params_stitch['regularization']['translation_factor'] = 0.005  # default: 0.005

# Export montage settings to
stitch_json = dir_project / '_jsons_montage' / stack / 'montage.json'
stitch_json.parent.mkdir(parents=True, exist_ok=True)
with stitch_json.open('w') as json_data:
    json.dump(params_stitch, json_data, indent=2)

# Check alignment parameters
print(stitch_json)
print('-'*len(stitch_json.as_posix()))
pprint(params_stitch)

/long_term_storage/skaracoban/data/test_data/20240222_RL019_hoechst_exocrine/_jsons_montage/EM_himag/montage.json
-----------------------------------------------------------------------------------------------------------------
{'close_stack': 'True',
 'first_section': 0.0,
 'hdf5_options': {'chunks_per_file': -1, 'output_dir': ''},
 'input_stack': {'client_scripts': '/home/catmaid/render/render-ws-java-client/src/main/scripts',
                 'collection_type': 'stack',
                 'db_interface': 'render',
                 'host': 'http://localhost',
                 'name': 'EM_himag',
                 'owner': 'skaracoban',
                 'port': 8081,
                 'project': '20240222_RL019_hoechst_exocrine'},
 'last_section': 0.0,
 'log_level': 'INFO',
 'matrix_assembly': {'cross_pt_weight': 1.0,
                     'depth': 2,
                     'inverse_dz': 'True',
                     'montage_pt_weight': 1.0,
                     'npts_max': 500,
            

### Compute optimal transformations with `BigFeta`

In [16]:
# Path to `BigFeta`
cwd = Path.cwd().as_posix()
dir_BigFeta = Path('/home/skaracoban/render_code/BigFeta/')

# Select json for stitching
stitch_json = dir_project / '_jsons_montage' / stack / 'montage.json'

# Call `BigFeta.BigFeta` process -- have to switch to BigFeta directory
os.chdir(dir_BigFeta.as_posix())
subprocess.run(['python', '-m', 'bigfeta.bigfeta', '--input_json', stitch_json.as_posix()])
os.chdir(cwd)

/home/skaracoban/miniconda3/envs/render/lib/python3.10/site-packages/argschema/utils.py:346: FutureWarning: '--transform_apply' is using old-style command-line syntax with each element as a separate argument. This will not be supported in argschema after 2.0. See http://argschema.readthedocs.io/en/master/user/intro.html#command-line-specification for details.
  warnings.warn(warn_msg, FutureWarning)
INFO:bigfeta.utils:
 loaded 144 tile specs from 1 zvalues in 0.1 sec using interface: render
INFO:__main__: A created in 0.5 seconds
INFO:__main__:
 solved in 0.1 sec
 precision [norm(Kx-Lm)/norm(Lm)] = 3.3e-16, 6.8e-16
 error     [norm(Ax-b)] = 979.498, 1190.968
 [mean(Ax) +/- std(Ax)] : 0.2 +/- 5.5, 2.3 +/- 6.2
 [mean(error mag) +/- std(error mag)] : 7.5 +/- 4.1
INFO:__main__:
 scales: 1.00 +/- 0.00, 1.00 +/- 0.00
INFO:bigfeta.utils:
ingesting results to http://localhost:8081 skaracoban__20240222_RL019_hoechst_exocrine__EM_himag_stitched
INFO:bigfeta.utils:render output is going to /dev/n

### Inspect

In [18]:
# Plot stacks
stacks = ["EM_himag", "EM_himag_stitched"]
plotting.plot_stacks(
    stacks,
    **params_render
)

  0%|          | 0/2 [00:00<?, ?it/s]

interactive(children=(IntSlider(value=0, description='z', max=0), IntSlider(value=33062, description='vmin', m…